# Loading Data and checking the IV

In [1]:
import pandas as pd
import numpy as np
from optbinning import OptimalBinning
pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", None)
def calculate_iv_nobinning(data, target):
    iv_dict = {}

    # Convert target column to numeric (assuming class 1 = Good (0), 2 = Bad (1))
    data[target] = data[target].replace({1: 0, 2: 1}).astype(int)

    total_bad = data[target].sum()
    total_good = len(data) - total_bad

    for col in data.columns:
        if col == target:
            continue
        grouped = data.groupby(col)[target].agg(['count', 'sum'])
        grouped.columns = ['total', 'bad']
        grouped['good'] = grouped['total'] - grouped['bad']

        grouped = grouped[(grouped['bad'] > 0) & (grouped['good'] > 0)]

        grouped['%bad'] = grouped['bad'] / total_bad
        grouped['%good'] = grouped['good'] / total_good

        grouped['WoE'] = np.log(grouped['%good'] / grouped['%bad']).replace({np.inf: 0, -np.inf: 0})
        grouped['IV'] = (grouped['%good'] - grouped['%bad']) * grouped['WoE']

        iv_value = grouped['IV'].sum()
        iv_dict[col] = iv_value

    return pd.DataFrame(iv_dict.items(), columns=['Feature', 'IV']).sort_values(by='IV', ascending=False)


# Read the dataset
data = pd.read_csv("german.data", delim_whitespace=True, header=None)

# Assign correct column names
column_names = [
    "Attribute1", "Attribute2", "Attribute3", "Attribute4", "Attribute5", 
    "Attribute6", "Attribute7", "Attribute8", "Attribute9", "Attribute10", 
    "Attribute11", "Attribute12", "Attribute13", "Attribute14", "Attribute15", 
    "Attribute16", "Attribute17", "Attribute18", "Attribute19", "Attribute20", 
    "class"
]
data.columns = column_names

# Compute IV values using the correct target column
iv_values = calculate_iv_nobinning(data, target="class")

# Display IV values
print(iv_values)


        Feature        IV
0    Attribute1  0.666012
1    Attribute2  0.309537
2    Attribute3  0.293234
12  Attribute13  0.257365
5    Attribute6  0.196010
3    Attribute4  0.169195
11  Attribute12  0.112638
6    Attribute7  0.086434
14  Attribute15  0.083293
13  Attribute14  0.057615
8    Attribute9  0.044671
19  Attribute20  0.043877
4    Attribute5  0.042108
9   Attribute10  0.032019
7    Attribute8  0.026322
15  Attribute16  0.013267
16  Attribute17  0.008763
18  Attribute19  0.006378
10  Attribute11  0.003589
17  Attribute18  0.000043


C:\Users\91828\AppData\Local\Temp\ipykernel_11640\3200366335.py:37: FutureWarning: The 'delim_whitespace' keyword in pd.read_csv is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead
  data = pd.read_csv("german.data", delim_whitespace=True, header=None)


In [2]:
data

,Attribute1,Attribute2,Attribute3,Attribute4,Attribute5,Attribute6,Attribute7,Attribute8,Attribute9,Attribute10,Attribute11,Attribute12,Attribute13,Attribute14,Attribute15,Attribute16,Attribute17,Attribute18,Attribute19,Attribute20,class
0,A11,6,A34,A43,1169,A65,A75,4,A93,A101,4,A121,67,A143,A152,2,A173,1,A192,A201,0
1,A12,48,A32,A43,5951,A61,A73,2,A92,A101,2,A121,22,A143,A152,1,A173,1,A191,A201,1
2,A14,12,A34,A46,2096,A61,A74,2,A93,A101,3,A121,49,A143,A152,1,A172,2,A191,A201,0
3,A11,42,A32,A42,7882,A61,A74,2,A93,A103,4,A122,45,A143,A153,1,A173,2,A191,A201,0
4,A11,24,A33,A40,4870,A61,A73,3,A93,A101,4,A124,53,A143,A153,2,A173,2,A191,A201,1
5,A14,36,A32,A46,9055,A65,A73,2,A93,A101,4,A124,35,A143,A153,1,A172,2,A192,A201,0
6,A14,24,A32,A42,2835,A63,A75,3,A93,A101,4,A122,53,A143,A152,1,A173,1,A191,A201,0
7,A12,36,A32,A41,6948,A61,A73,2,A93,A101,2,A123,35,A143,A151,1,A174,1,A192,A201,0
8,A14,12,A32,A43,3059,A64,A74,2,A91,A101,4,A121,61,A143,A152,1,A172,1,A191,A201,0
9,A12,30,A34,A40,5234,A61,A71,4,A94,A101,2,A123,28,A143,A152,2,A174,1,A191,A201,1


In [3]:
iv_values.loc[(iv_values["IV"] > 0.019999999) & (iv_values["IV"] < 0.7)]

,Feature,IV
0,Attribute1,0.666012
1,Attribute2,0.309537
2,Attribute3,0.293234
12,Attribute13,0.257365
5,Attribute6,0.196010
3,Attribute4,0.169195
11,Attribute12,0.112638
6,Attribute7,0.086434
14,Attribute15,0.083293
13,Attribute14,0.057615


# Binning data

In [4]:
coln = ["class","Attribute2","Attribute5","Attribute8","Attribute13"]
num_col = data[coln]
n = num_col.copy()

In [5]:
num_col.head()

,class,Attribute2,Attribute5,Attribute8,Attribute13
0,0,6,1169,4,67
1,1,48,5951,2,22
2,0,12,2096,2,49
3,0,42,7882,2,45
4,1,24,4870,3,53


In [6]:
variable = 'Attribute2'
x = num_col[variable].values
y = num_col["class"]
optb = OptimalBinning(name=variable, dtype="numerical", solver="cp", max_n_bins=6)
optb.fit(x, y)

OptimalBinning(max_n_bins=6, name='Attribute2')

In [7]:
binning_table = optb.binning_table
Attribute2_table = binning_table.build()
Attribute2_table

,Bin,Count,Count (%),Non-event,Event,Event rate,WoE,IV,JS
0,"(-inf, 8.50)",94,0.094,84,10,0.106383,1.280934,0.111014,0.013000
1,"[8.50, 11.50)",86,0.086,69,17,0.197674,0.553595,0.023198,0.002863
2,"[11.50, 15.50)",251,0.251,189,62,0.247012,0.267315,0.016930,0.002110
3,"[15.50, 34.50)",399,0.399,270,129,0.323308,-0.108688,0.004813,0.000601
4,"[34.50, 43.50)",100,0.100,58,42,0.420000,-0.524524,0.029973,0.003704
5,"[43.50, inf)",70,0.070,30,40,0.571429,-1.13498,0.102689,0.012189
6,Special,0,0.000,0,0,0.000000,0.0,0.000000,0.000000
7,Missing,0,0.000,0,0,0.000000,0.0,0.000000,0.000000
Totals,,1000,1.000,700,300,0.300000,,0.288617,0.034467


In [8]:
def categorize_annual_amount(value):
    if value < 8.50:
        return '(-inf, 8.50)'
    elif 8.50 <= value < 11.50:
        return '[8.50, 11.50)'
    elif 11.50 <= value < 15.50:
        return '[11.50, 15.50)'
    elif 15.50 <= value < 34.50:
        return '[15.50, 34.50)'
    elif 34.50 <= value < 43.50:
        return '[34.50, 43.50)'
    elif value >= 43.50:
        return '[43.50, inf)'
    else:
        return 'Missing'

In [9]:
num_col[f"{variable}_BINNED"] = num_col['Attribute2'].apply(lambda x: categorize_annual_amount(x) if pd.notna(x) else 'Missing')
num_col.head()

C:\Users\91828\AppData\Local\Temp\ipykernel_11640\3241208519.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  num_col[f"{variable}_BINNED"] = num_col['Attribute2'].apply(lambda x: categorize_annual_amount(x) if pd.notna(x) else 'Missing')


,class,Attribute2,Attribute5,Attribute8,Attribute13,Attribute2_BINNED
0,0,6,1169,4,67,"(-inf, 8.50)"
1,1,48,5951,2,22,"[43.50, inf)"
2,0,12,2096,2,49,"[11.50, 15.50)"
3,0,42,7882,2,45,"[34.50, 43.50)"
4,1,24,4870,3,53,"[15.50, 34.50)"


In [10]:
Attribute2_table.insert(
    Attribute2_table.columns.get_loc("Bin") + 1, 
    "Column", 
    "Attribute2_BINNED"
)
Attribute2_table

,Bin,Column,Count,Count (%),Non-event,Event,Event rate,WoE,IV,JS
0,"(-inf, 8.50)",Attribute2_BINNED,94,0.094,84,10,0.106383,1.280934,0.111014,0.013000
1,"[8.50, 11.50)",Attribute2_BINNED,86,0.086,69,17,0.197674,0.553595,0.023198,0.002863
2,"[11.50, 15.50)",Attribute2_BINNED,251,0.251,189,62,0.247012,0.267315,0.016930,0.002110
3,"[15.50, 34.50)",Attribute2_BINNED,399,0.399,270,129,0.323308,-0.108688,0.004813,0.000601
4,"[34.50, 43.50)",Attribute2_BINNED,100,0.100,58,42,0.420000,-0.524524,0.029973,0.003704
5,"[43.50, inf)",Attribute2_BINNED,70,0.070,30,40,0.571429,-1.13498,0.102689,0.012189
6,Special,Attribute2_BINNED,0,0.000,0,0,0.000000,0.0,0.000000,0.000000
7,Missing,Attribute2_BINNED,0,0.000,0,0,0.000000,0.0,0.000000,0.000000
Totals,,Attribute2_BINNED,1000,1.000,700,300,0.300000,,0.288617,0.034467


In [11]:
variable = 'Attribute5'
x = num_col[variable].values
y = num_col["class"]
optb = OptimalBinning(name=variable, dtype="numerical", solver="cp", max_n_bins=6)
optb.fit(x, y)

OptimalBinning(max_n_bins=6, name='Attribute5')

In [12]:
binning_table = optb.binning_table
Attribute5_table = binning_table.build()
Attribute5_table

,Bin,Count,Count (%),Non-event,Event,Event rate,WoE,IV,JS
0,"(-inf, 954.50)",107,0.107,72,35,0.327103,-0.12598,0.001740,0.000217
1,"[954.50, 1373.00)",146,0.146,101,45,0.308219,-0.03884,0.000222,0.000028
2,"[1373.00, 3446.50)",427,0.427,323,104,0.243560,0.285964,0.032818,0.004088
3,"[3446.50, 3913.50)",60,0.060,55,5,0.083333,1.550597,0.095989,0.010925
4,"[3913.50, 7839.50)",185,0.185,115,70,0.378378,-0.350861,0.024226,0.003013
5,"[7839.50, inf)",75,0.075,34,41,0.546667,-1.034509,0.091135,0.010910
6,Special,0,0.000,0,0,0.000000,0.0,0.000000,0.000000
7,Missing,0,0.000,0,0,0.000000,0.0,0.000000,0.000000
Totals,,1000,1.000,700,300,0.300000,,0.246130,0.029181


In [13]:
def categorize_annual_amount(value):
    if value < 954.50:
        return '(-inf, 954.50)'
    elif 954.50 <= value < 1373.00:
        return '[954.50, 1373.00)'
    elif 1373.00 <= value < 3446.50:
        return '[1373.00, 3446.50)'
    elif 3446.50 <= value < 3913.50:
        return '[3446.50, 3913.50)'
    elif 3913.50 <= value < 7839.50:
        return '[3913.50, 7839.50)'
    elif value >=7839.50:
        return '[7839.50, inf)'
    else:
        return 'Missing'

In [14]:
num_col[f"{variable}_BINNED"] = num_col['Attribute5'].apply(lambda x: categorize_annual_amount(x) if pd.notna(x) else 'Missing')
num_col.head()

C:\Users\91828\AppData\Local\Temp\ipykernel_11640\572181244.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  num_col[f"{variable}_BINNED"] = num_col['Attribute5'].apply(lambda x: categorize_annual_amount(x) if pd.notna(x) else 'Missing')


,class,Attribute2,Attribute5,Attribute8,Attribute13,Attribute2_BINNED,Attribute5_BINNED
0,0,6,1169,4,67,"(-inf, 8.50)","[954.50, 1373.00)"
1,1,48,5951,2,22,"[43.50, inf)","[3913.50, 7839.50)"
2,0,12,2096,2,49,"[11.50, 15.50)","[1373.00, 3446.50)"
3,0,42,7882,2,45,"[34.50, 43.50)","[7839.50, inf)"
4,1,24,4870,3,53,"[15.50, 34.50)","[3913.50, 7839.50)"


In [15]:
Attribute5_table.insert(
    Attribute5_table.columns.get_loc("Bin") + 1, 
    "Column", 
    "Attribute5_BINNED"
)
Attribute5_table

,Bin,Column,Count,Count (%),Non-event,Event,Event rate,WoE,IV,JS
0,"(-inf, 954.50)",Attribute5_BINNED,107,0.107,72,35,0.327103,-0.12598,0.001740,0.000217
1,"[954.50, 1373.00)",Attribute5_BINNED,146,0.146,101,45,0.308219,-0.03884,0.000222,0.000028
2,"[1373.00, 3446.50)",Attribute5_BINNED,427,0.427,323,104,0.243560,0.285964,0.032818,0.004088
3,"[3446.50, 3913.50)",Attribute5_BINNED,60,0.060,55,5,0.083333,1.550597,0.095989,0.010925
4,"[3913.50, 7839.50)",Attribute5_BINNED,185,0.185,115,70,0.378378,-0.350861,0.024226,0.003013
5,"[7839.50, inf)",Attribute5_BINNED,75,0.075,34,41,0.546667,-1.034509,0.091135,0.010910
6,Special,Attribute5_BINNED,0,0.000,0,0,0.000000,0.0,0.000000,0.000000
7,Missing,Attribute5_BINNED,0,0.000,0,0,0.000000,0.0,0.000000,0.000000
Totals,,Attribute5_BINNED,1000,1.000,700,300,0.300000,,0.246130,0.029181


In [16]:
variable = 'Attribute8'
x = num_col[variable].values
y = num_col["class"]
optb = OptimalBinning(name=variable, dtype="numerical", solver="cp", max_n_bins=6)
optb.fit(x, y)

OptimalBinning(max_n_bins=6, name='Attribute8')

In [17]:
binning_table = optb.binning_table
Attribute8_table = binning_table.build()
Attribute8_table

,Bin,Count,Count (%),Non-event,Event,Event rate,WoE,IV,JS
0,"(-inf, 1.50)",136,0.136,102,34,0.250000,0.251314,0.008138,0.001015
1,"[1.50, 2.50)",231,0.231,169,62,0.268398,0.155466,0.005404,0.000675
2,"[2.50, 3.50)",157,0.157,112,45,0.286624,0.064539,0.000645,0.000081
3,"[3.50, inf)",476,0.476,317,159,0.334034,-0.1573,0.012135,0.001515
4,Special,0,0.000,0,0,0.000000,0.0,0.000000,0.000000
5,Missing,0,0.000,0,0,0.000000,0.0,0.000000,0.000000
Totals,,1000,1.000,700,300,0.300000,,0.026322,0.003285


In [18]:
def categorize_annual_amount(value):
    if value < 1.50:
        return '(-inf, 1.50)'
    elif 1.50 <= value < 2.50:
        return '[1.50, 2.50)'
    elif 2.50 <= value < 3.50:
        return '[2.50, 3.50)'
    elif value >= 3.50:
        return '[3.50, inf)'
    else:
        return 'Missing'
num_col[f"{variable}_BINNED"] = num_col['Attribute8'].apply(lambda x: categorize_annual_amount(x) if pd.notna(x) else 'Missing')
num_col.head()

C:\Users\91828\AppData\Local\Temp\ipykernel_11640\327724859.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  num_col[f"{variable}_BINNED"] = num_col['Attribute8'].apply(lambda x: categorize_annual_amount(x) if pd.notna(x) else 'Missing')


,class,Attribute2,Attribute5,Attribute8,Attribute13,Attribute2_BINNED,Attribute5_BINNED,Attribute8_BINNED
0,0,6,1169,4,67,"(-inf, 8.50)","[954.50, 1373.00)","[3.50, inf)"
1,1,48,5951,2,22,"[43.50, inf)","[3913.50, 7839.50)","[1.50, 2.50)"
2,0,12,2096,2,49,"[11.50, 15.50)","[1373.00, 3446.50)","[1.50, 2.50)"
3,0,42,7882,2,45,"[34.50, 43.50)","[7839.50, inf)","[1.50, 2.50)"
4,1,24,4870,3,53,"[15.50, 34.50)","[3913.50, 7839.50)","[2.50, 3.50)"


In [19]:
Attribute8_table.insert(
    Attribute8_table.columns.get_loc("Bin") + 1, 
    "Column", 
    "Attribute8_BINNED"
)
Attribute8_table

,Bin,Column,Count,Count (%),Non-event,Event,Event rate,WoE,IV,JS
0,"(-inf, 1.50)",Attribute8_BINNED,136,0.136,102,34,0.250000,0.251314,0.008138,0.001015
1,"[1.50, 2.50)",Attribute8_BINNED,231,0.231,169,62,0.268398,0.155466,0.005404,0.000675
2,"[2.50, 3.50)",Attribute8_BINNED,157,0.157,112,45,0.286624,0.064539,0.000645,0.000081
3,"[3.50, inf)",Attribute8_BINNED,476,0.476,317,159,0.334034,-0.1573,0.012135,0.001515
4,Special,Attribute8_BINNED,0,0.000,0,0,0.000000,0.0,0.000000,0.000000
5,Missing,Attribute8_BINNED,0,0.000,0,0,0.000000,0.0,0.000000,0.000000
Totals,,Attribute8_BINNED,1000,1.000,700,300,0.300000,,0.026322,0.003285


In [20]:
variable = 'Attribute13'
x = num_col[variable].values
y = num_col["class"]
optb = OptimalBinning(name=variable, dtype="numerical", solver="cp", max_n_bins=6)
optb.fit(x, y)

OptimalBinning(max_n_bins=6, name='Attribute13')

In [21]:
binning_table = optb.binning_table
Attribute13_table = binning_table.build()
Attribute13_table

,Bin,Count,Count (%),Non-event,Event,Event rate,WoE,IV,JS
0,"(-inf, 25.50)",190,0.190,110,80,0.421053,-0.528844,0.057921,0.007157
1,"[25.50, 34.50)",358,0.358,246,112,0.312849,-0.060465,0.001324,0.000166
2,"[34.50, 36.50)",79,0.079,67,12,0.151899,0.872488,0.048610,0.005891
3,"[36.50, 38.50)",53,0.053,41,12,0.226415,0.381368,0.007083,0.000880
4,"[38.50, 52.50)",224,0.224,169,55,0.245536,0.275268,0.015992,0.001993
5,"[52.50, inf)",96,0.096,67,29,0.302083,-0.009901,0.000009,0.000001
6,Special,0,0.000,0,0,0.000000,0.0,0.000000,0.000000
7,Missing,0,0.000,0,0,0.000000,0.0,0.000000,0.000000
Totals,,1000,1.000,700,300,0.300000,,0.130939,0.016087


In [22]:
def categorize_annual_amount(value):
    if value < 25.50:
        return '(-inf, 25.50)'
    elif 25.50 <= value < 34.50:
        return '[25.50, 34.50)'
    elif 34.50 <= value < 36.50:
        return '[34.50, 36.50)'
    elif 36.50 <= value < 38.50:
        return '[36.50, 38.50)'
    elif 38.50 <= value < 52.50:
        return '[38.50, 52.50)'
    elif value >= 52.50:
        return '[52.50, inf)'
    else:
        return 'Missing'

num_col[f"{variable}_BINNED"] = num_col['Attribute13'].apply(lambda x: categorize_annual_amount(x) if pd.notna(x) else 'Missing')
num_col.head()

C:\Users\91828\AppData\Local\Temp\ipykernel_11640\2485645610.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  num_col[f"{variable}_BINNED"] = num_col['Attribute13'].apply(lambda x: categorize_annual_amount(x) if pd.notna(x) else 'Missing')


,class,Attribute2,Attribute5,Attribute8,Attribute13,Attribute2_BINNED,Attribute5_BINNED,Attribute8_BINNED,Attribute13_BINNED
0,0,6,1169,4,67,"(-inf, 8.50)","[954.50, 1373.00)","[3.50, inf)","[52.50, inf)"
1,1,48,5951,2,22,"[43.50, inf)","[3913.50, 7839.50)","[1.50, 2.50)","(-inf, 25.50)"
2,0,12,2096,2,49,"[11.50, 15.50)","[1373.00, 3446.50)","[1.50, 2.50)","[38.50, 52.50)"
3,0,42,7882,2,45,"[34.50, 43.50)","[7839.50, inf)","[1.50, 2.50)","[38.50, 52.50)"
4,1,24,4870,3,53,"[15.50, 34.50)","[3913.50, 7839.50)","[2.50, 3.50)","[52.50, inf)"


In [23]:
Attribute13_table.insert(
    Attribute13_table.columns.get_loc("Bin") + 1, 
    "Column", 
    "Attribute13_BINNED"
)
Attribute13_table

,Bin,Column,Count,Count (%),Non-event,Event,Event rate,WoE,IV,JS
0,"(-inf, 25.50)",Attribute13_BINNED,190,0.190,110,80,0.421053,-0.528844,0.057921,0.007157
1,"[25.50, 34.50)",Attribute13_BINNED,358,0.358,246,112,0.312849,-0.060465,0.001324,0.000166
2,"[34.50, 36.50)",Attribute13_BINNED,79,0.079,67,12,0.151899,0.872488,0.048610,0.005891
3,"[36.50, 38.50)",Attribute13_BINNED,53,0.053,41,12,0.226415,0.381368,0.007083,0.000880
4,"[38.50, 52.50)",Attribute13_BINNED,224,0.224,169,55,0.245536,0.275268,0.015992,0.001993
5,"[52.50, inf)",Attribute13_BINNED,96,0.096,67,29,0.302083,-0.009901,0.000009,0.000001
6,Special,Attribute13_BINNED,0,0.000,0,0,0.000000,0.0,0.000000,0.000000
7,Missing,Attribute13_BINNED,0,0.000,0,0,0.000000,0.0,0.000000,0.000000
Totals,,Attribute13_BINNED,1000,1.000,700,300,0.300000,,0.130939,0.016087


In [24]:
numcol = pd.concat([Attribute2_table,Attribute5_table,Attribute8_table,Attribute13_table])
numcol

,Bin,Column,Count,Count (%),Non-event,Event,Event rate,WoE,IV,JS
0,"(-inf, 8.50)",Attribute2_BINNED,94,0.094,84,10,0.106383,1.280934,0.111014,0.013000
1,"[8.50, 11.50)",Attribute2_BINNED,86,0.086,69,17,0.197674,0.553595,0.023198,0.002863
2,"[11.50, 15.50)",Attribute2_BINNED,251,0.251,189,62,0.247012,0.267315,0.016930,0.002110
3,"[15.50, 34.50)",Attribute2_BINNED,399,0.399,270,129,0.323308,-0.108688,0.004813,0.000601
4,"[34.50, 43.50)",Attribute2_BINNED,100,0.100,58,42,0.420000,-0.524524,0.029973,0.003704
5,"[43.50, inf)",Attribute2_BINNED,70,0.070,30,40,0.571429,-1.13498,0.102689,0.012189
6,Special,Attribute2_BINNED,0,0.000,0,0,0.000000,0.0,0.000000,0.000000
7,Missing,Attribute2_BINNED,0,0.000,0,0,0.000000,0.0,0.000000,0.000000
Totals,,Attribute2_BINNED,1000,1.000,700,300,0.300000,,0.288617,0.034467
0,"(-inf, 954.50)",Attribute5_BINNED,107,0.107,72,35,0.327103,-0.12598,0.001740,0.000217


In [25]:
print(num_col.columns)

Index(['class', 'Attribute2', 'Attribute5', 'Attribute8', 'Attribute13',
       'Attribute2_BINNED', 'Attribute5_BINNED', 'Attribute8_BINNED',
       'Attribute13_BINNED'],
      dtype='object')


In [26]:
print(numcol)

                       Bin              Column  Count  Count (%)  Non-event  \
0             (-inf, 8.50)   Attribute2_BINNED     94      0.094         84   
1            [8.50, 11.50)   Attribute2_BINNED     86      0.086         69   
2           [11.50, 15.50)   Attribute2_BINNED    251      0.251        189   
3           [15.50, 34.50)   Attribute2_BINNED    399      0.399        270   
4           [34.50, 43.50)   Attribute2_BINNED    100      0.100         58   
5             [43.50, inf)   Attribute2_BINNED     70      0.070         30   
6                  Special   Attribute2_BINNED      0      0.000          0   
7                  Missing   Attribute2_BINNED      0      0.000          0   
Totals                       Attribute2_BINNED   1000      1.000        700   
0           (-inf, 954.50)   Attribute5_BINNED    107      0.107         72   
1        [954.50, 1373.00)   Attribute5_BINNED    146      0.146        101   
2       [1373.00, 3446.50)   Attribute5_BINNED    42

In [27]:
n = num_col.copy()
print(n.columns)

Index(['class', 'Attribute2', 'Attribute5', 'Attribute8', 'Attribute13',
       'Attribute2_BINNED', 'Attribute5_BINNED', 'Attribute8_BINNED',
       'Attribute13_BINNED'],
      dtype='object')


In [28]:
woe_mappings = {}

attributes = ["Attribute2_BINNED", "Attribute5_BINNED", "Attribute8_BINNED", "Attribute13_BINNED"]

for col in attributes:
    col_woe_mapping = numcol[numcol["Column"] == col][["Bin", "WoE"]]
    woe_mappings[col] = dict(zip(col_woe_mapping["Bin"], col_woe_mapping["WoE"]))

for col in attributes:
    n[col] = num_col[col].apply(lambda x: next((woe for bin_range, woe in woe_mappings[col].items() if str(x) in str(bin_range)), None))

n.head()

,class,Attribute2,Attribute5,Attribute8,Attribute13,Attribute2_BINNED,Attribute5_BINNED,Attribute8_BINNED,Attribute13_BINNED
0,0,6,1169,4,67,1.280934,-0.038840,-0.157300,-0.009901
1,1,48,5951,2,22,-1.134980,-0.350861,0.155466,-0.528844
2,0,12,2096,2,49,0.267315,0.285964,0.155466,0.275268
3,0,42,7882,2,45,-0.524524,-1.034509,0.155466,0.275268
4,1,24,4870,3,53,-0.108688,-0.350861,0.064539,-0.009901


In [29]:
cat = ["class","Attribute1","Attribute3","Attribute6","Attribute4","Attribute12","Attribute7","Attribute15","Attribute14",
       "Attribute9","Attribute20","Attribute10"]
df = data[cat]

In [30]:
df.head()

,class,Attribute1,Attribute3,Attribute6,Attribute4,Attribute12,Attribute7,Attribute15,Attribute14,Attribute9,Attribute20,Attribute10
0,0,A11,A34,A65,A43,A121,A75,A152,A143,A93,A201,A101
1,1,A12,A32,A61,A43,A121,A73,A152,A143,A92,A201,A101
2,0,A14,A34,A61,A46,A121,A74,A152,A143,A93,A201,A101
3,0,A11,A32,A61,A42,A122,A74,A153,A143,A93,A201,A103
4,1,A11,A33,A61,A40,A124,A73,A153,A143,A93,A201,A101


In [31]:
import numpy as np
import pandas as pd

# List of categorical columns for which we need WoE calculations
categorical_cols = ["Attribute1", "Attribute3", "Attribute6", "Attribute4", "Attribute12", 
                    "Attribute7", "Attribute15", "Attribute14", "Attribute9", "Attribute20", "Attribute10"]

# Creating an empty list to store results
woe_tables = []

# Iterate through each categorical column
for col in categorical_cols:
    # Group by column to get total count per category
    attr_table = df.groupby(col)["class"].count().reset_index()
    attr_table.columns = ["Bin", "Count"]

    # Get event (class = 1) and non-event (class = 0) counts
    event_count = df[df["class"] == 1].groupby(col)["class"].count().reset_index()
    event_count.columns = ["Bin", "Event"]
    
    non_event_count = df[df["class"] == 0].groupby(col)["class"].count().reset_index()
    non_event_count.columns = ["Bin", "Non-event"]

    # Merge event and non-event counts
    attr_table = attr_table.merge(event_count, on="Bin", how="left").merge(non_event_count, on="Bin", how="left")
    
    # Fill NaN values with 0 (if any category has missing event/non-event counts)
    attr_table.fillna(0, inplace=True)

    # Add column name
    attr_table.insert(attr_table.columns.get_loc("Bin") + 1, "Column", col)

    # Calculate Count (%)
    total_count = attr_table["Count"].sum()
    attr_table["Count (%)"] = (attr_table["Count"] / total_count) * 100

    # Calculate Event Rate
    attr_table["Event rate"] = attr_table["Event"] / attr_table["Count"]

    # Calculate WoE (Weight of Evidence)
    total_event = attr_table["Event"].sum()
    total_non_event = attr_table["Non-event"].sum()

    attr_table["Event %"] = attr_table["Event"] / total_event
    attr_table["Non-event %"] = attr_table["Non-event"] / total_non_event

    # Handling division by zero in WoE calculation
    attr_table["WoE"] = np.log((attr_table["Event %"] / attr_table["Non-event %"]).replace({0: np.nan}))  

    # Drop intermediate columns
    attr_table.drop(columns=["Event %", "Non-event %"], inplace=True)

    # Append result to list
    woe_tables.append(attr_table)

# Concatenating all attribute tables into one final table
final_woe_table = pd.concat(woe_tables, ignore_index=True)

# Display final WoE table
final_woe_table


,Bin,Column,Count,Event,Non-event,Count (%),Event rate,WoE
0,A11,Attribute1,274,135,139,27.4,0.492701,0.818099
1,A12,Attribute1,269,105,164,26.9,0.390335,0.401392
2,A13,Attribute1,63,14,49,6.3,0.222222,-0.405465
3,A14,Attribute1,394,46,348,39.4,0.116751,-1.176263
4,A30,Attribute3,40,25,15,4.0,0.625000,1.358123
5,A31,Attribute3,49,28,21,4.9,0.571429,1.134980
6,A32,Attribute3,530,169,361,53.0,0.318868,0.088319
7,A33,Attribute3,88,28,60,8.8,0.318182,0.085158
8,A34,Attribute3,293,50,243,29.3,0.170648,-0.733741
9,A61,Attribute6,603,217,386,60.3,0.359867,0.271358


In [32]:
woe_mappings = {}

for col in categorical_cols:
    col_woe_mapping = final_woe_table[final_woe_table["Column"] == col][["Bin", "WoE"]]
    
    woe_mappings[col] = dict(zip(col_woe_mapping["Bin"], col_woe_mapping["WoE"]))

df_woe = df.copy()

for col in categorical_cols:
    df_woe[col] = df_woe[col].map(woe_mappings[col])

df_woe.head()

,class,Attribute1,Attribute3,Attribute6,Attribute4,Attribute12,Attribute7,Attribute15,Attribute14,Attribute9,Attribute20,Attribute10
0,0,0.818099,-0.733741,-0.704246,-0.410063,-0.461035,-0.235566,-0.194156,-0.121179,-0.165548,0.034867,-0.000525
1,1,0.401392,0.088319,0.271358,-0.410063,-0.461035,0.032103,-0.194156,-0.121179,0.235341,0.034867,-0.000525
2,0,-1.176263,-0.733741,0.271358,0.606136,-0.461035,-0.394415,-0.194156,-0.121179,-0.165548,0.034867,-0.000525
3,0,0.818099,0.088319,0.271358,0.095557,0.028573,-0.394415,0.472604,-0.121179,-0.165548,0.034867,-0.587787
4,1,0.818099,0.085158,0.271358,0.359200,0.586082,0.032103,0.472604,-0.121179,-0.165548,0.034867,-0.000525


In [33]:
Final_data = df_woe.merge(n, on = "class", how = "left")

In [34]:
Final_data.head()

,class,Attribute1,Attribute3,Attribute6,Attribute4,Attribute12,Attribute7,Attribute15,Attribute14,Attribute9,Attribute20,Attribute10,Attribute2,Attribute5,Attribute8,Attribute13,Attribute2_BINNED,Attribute5_BINNED,Attribute8_BINNED,Attribute13_BINNED
0,0,0.818099,-0.733741,-0.704246,-0.410063,-0.461035,-0.235566,-0.194156,-0.121179,-0.165548,0.034867,-0.000525,6,1169,4,67,1.280934,-0.038840,-0.157300,-0.009901
1,0,0.818099,-0.733741,-0.704246,-0.410063,-0.461035,-0.235566,-0.194156,-0.121179,-0.165548,0.034867,-0.000525,12,2096,2,49,0.267315,0.285964,0.155466,0.275268
2,0,0.818099,-0.733741,-0.704246,-0.410063,-0.461035,-0.235566,-0.194156,-0.121179,-0.165548,0.034867,-0.000525,42,7882,2,45,-0.524524,-1.034509,0.155466,0.275268
3,0,0.818099,-0.733741,-0.704246,-0.410063,-0.461035,-0.235566,-0.194156,-0.121179,-0.165548,0.034867,-0.000525,36,9055,2,35,-0.524524,-1.034509,0.155466,0.872488
4,0,0.818099,-0.733741,-0.704246,-0.410063,-0.461035,-0.235566,-0.194156,-0.121179,-0.165548,0.034867,-0.000525,24,2835,3,53,-0.108688,0.285964,0.064539,-0.009901


In [35]:
Selected_variables = ["class","Attribute1","Attribute2_BINNED","Attribute3","Attribute4","Attribute6","Attribute7",
                      "Attribute8_BINNED","Attribute9","Attribute10","Attribute12","Attribute15","Attribute20"]
sel_var = Final_data[Selected_variables]

In [36]:
new_data = sel_var.to_csv('EDA_Variables.csv', index=False)

In [38]:
data = pd.read_csv('EDA_Variables.csv')